In [16]:
# Importando as bibliotecas necessárias 
import os # Permite acesso as variáveis de ambiente 
from dotenv import load_dotenv # Carrega os dados das variáveis de ambiente 
from langchain_google_genai import ChatGoogleGenerativeAI # Carrega o modelo de IA generativa 

# Carrega as variáveis do arquivo .env
load_dotenv()

# Importa a chave
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

In [17]:
# Prompt do sistema 
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [18]:
# Importando as bibliotecas para a validação 
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

# Classe de validação da saída do modelo no formato JSON 
class TriagemOut(BaseModel): 
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"] # Aceita apenas os valores listados 
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list) # Cria uma lista com um valor padrão (lista vazia)

In [19]:
# Instanciação do modelo de linguagem do Gemini
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0, # Configura a criatividade do modelo
    api_key=GOOGLE_API_KEY)

In [20]:
# Importando as funções que criam a cadeia entre o modelo e a classe de validação
from langchain_core.messages import SystemMessage, HumanMessage

# Conecta o modelo de linguagem com a classe de validação com saída estruturada no formato JSON
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

# Criando a função orquestradora
def triagem(mensagem: str) -> Dict: # Executa a cadeia de triagem e retorna a saída validada com um dicionário
    saida: TriagemOut = triagem_chain.invoke([SystemMessage(content=TRIAGEM_PROMPT), # Contém instruções da persona que o modelo deve assumir, no prompt do sistema 
                                              HumanMessage(content=mensagem)]) # Contém a pergunta do usuário
    return saida.model_dump() # Converte o objeto pydantic para um dicionário

In [21]:
testes = ["Posso reembolsar a internet?", 
          "Quero mais 5 dias de trabalho remoto. Como faço?", 
          "Posso reembolsar cursos e/ou treinamentos?", 
          "Quantas capivaras tem no Rio Araguaia?"]

In [ ]:
# Itera sobre a lista de perguntas e ativa a função orquestradora com uma pergunta como parâmetro
for teste in testes: 
    print(f"Pergunta: {teste}\n Resposta -> {triagem(teste)}\n")

Pergunta: Posso reembolsar a internet?
 Resposta -> {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

